In [15]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN

In [2]:
metapath = []
vectors = []
with open('../network_embedding/embeddings_output/metapath_vectors.txt') as f:
    for line in f:
        splits = line.split()
        if len(splits) == 2:
            print('metapath: %s, embedding_size: %s' % (splits[0], splits[1]))
            continue
        metapath.append(splits[0])
        vectors.append(list(map(float, splits[1:])))

metapath: 30, embedding_size: 128


In [31]:
kmeans = KMeans(n_clusters=5).fit(vectors)
kmeans.labels_

array([4, 3, 0, 2, 4, 3, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [10]:
metapath

['U-A',
 'A-U',
 'A-U,U-A',
 'U-A,A-U',
 'U-A,A-U,U-A',
 'A-U,U-A,A-U',
 'A-U,U-A,A-U,U-A',
 'U-A,A-U,U-A,A-U',
 'C-A',
 'C-A,A-U',
 'C-A,A-U,U-A',
 'A-C',
 'A-C,C-A',
 'C-A,A-U,U-A,A-U',
 'U-A,A-C',
 'U-A,A-C,C-A',
 'A-U,U-A,A-C',
 'A-U,U-A,A-C,C-A',
 'A-C,C-A,A-U',
 'A-C,C-A,A-U,U-A',
 'U-A,A-U,U-A,A-C',
 'U-A,A-C,C-A,A-U',
 'C-A,A-C',
 'C-A,A-C,C-A',
 'A-C,C-A,A-C',
 'A-C,C-A,A-C,C-A',
 'C-A,A-C,C-A,A-U',
 'U-A,A-C,C-A,A-C',
 'C-A,A-U,U-A,A-C',
 'C-A,A-C,C-A,A-C']

In [23]:
cluster = DBSCAN(eps=3, min_samples=2).fit(vectors)
cluster.labels_

array([-1, -1,  0, -1, -1, -1,  0, -1,  1,  2,  1, -1,  2,  2,  3,  3, -1,
        2,  2,  2,  3,  3,  2,  2,  2,  2,  2,  2,  2,  2])

In [28]:
def match(paths, labels):
    d = {}
    for i in range(len(paths)):
        path = paths[i]
        label = labels[i]
        if label not in d:
            d[label] = []
        d[label].append(path)
    for k, v in d.items():
        print(k, ':', v)

In [29]:
match(metapath, cluster.labels_)

-1 : ['U-A', 'A-U', 'U-A,A-U', 'U-A,A-U,U-A', 'A-U,U-A,A-U', 'U-A,A-U,U-A,A-U', 'A-C', 'A-U,U-A,A-C']
0 : ['A-U,U-A', 'A-U,U-A,A-U,U-A']
1 : ['C-A', 'C-A,A-U,U-A']
2 : ['C-A,A-U', 'A-C,C-A', 'C-A,A-U,U-A,A-U', 'A-U,U-A,A-C,C-A', 'A-C,C-A,A-U', 'A-C,C-A,A-U,U-A', 'C-A,A-C', 'C-A,A-C,C-A', 'A-C,C-A,A-C', 'A-C,C-A,A-C,C-A', 'C-A,A-C,C-A,A-U', 'U-A,A-C,C-A,A-C', 'C-A,A-U,U-A,A-C', 'C-A,A-C,C-A,A-C']
3 : ['U-A,A-C', 'U-A,A-C,C-A', 'U-A,A-U,U-A,A-C', 'U-A,A-C,C-A,A-U']


In [32]:
match(metapath, kmeans.labels_)

4 : ['U-A', 'U-A,A-U,U-A']
3 : ['A-U', 'A-U,U-A,A-U']
0 : ['A-U,U-A', 'A-U,U-A,A-U,U-A']
2 : ['U-A,A-U', 'U-A,A-U,U-A,A-U']
1 : ['C-A', 'C-A,A-U', 'C-A,A-U,U-A', 'A-C', 'A-C,C-A', 'C-A,A-U,U-A,A-U', 'U-A,A-C', 'U-A,A-C,C-A', 'A-U,U-A,A-C', 'A-U,U-A,A-C,C-A', 'A-C,C-A,A-U', 'A-C,C-A,A-U,U-A', 'U-A,A-U,U-A,A-C', 'U-A,A-C,C-A,A-U', 'C-A,A-C', 'C-A,A-C,C-A', 'A-C,C-A,A-C', 'A-C,C-A,A-C,C-A', 'C-A,A-C,C-A,A-U', 'U-A,A-C,C-A,A-C', 'C-A,A-U,U-A,A-C', 'C-A,A-C,C-A,A-C']
